## Decryption
### Requires access to the "locked" files, and to the corresponding key.

In [14]:
!dir

 Volume in drive C is Acer
 Volume Serial Number is 56B0-9870

 Directory of C:\Downloads\BYOD\DataLocker\Encryption-Schemas-with-Zip-Manipulation

03/02/2021  01:48 PM    <DIR>          .
03/02/2021  01:48 PM    <DIR>          ..
12/13/2019  06:50 PM             1,348 .gitignore
03/02/2021  01:47 PM    <DIR>          .ipynb_checkpoints
03/02/2021  12:33 PM             4,501 Decrypt.ipynb
03/02/2021  01:48 PM               168 Download.CSV
03/02/2021  01:47 PM               109 Download.CSV.key
03/02/2021  01:47 PM               176 Download.CSV.locked
03/02/2021  01:48 PM               216 Download.zip
03/02/2021  01:48 PM               113 Download.zip.key
03/02/2021  01:48 PM               224 Download.zip.locked
03/02/2021  01:46 PM            10,389 index.html
03/02/2021  09:56 AM             1,098 LICENSE
03/02/2021  01:02 PM               547 manifest.webmanifest
03/02/2021  11:38 AM               515 README.md
03/09/2020  08:23 AM                54 robots.txt
12/30/2019  09:30 

In [15]:
# So, we have our Download.CSV.locked, and Download.CSV.key

In [16]:
#!pip install pycryptodome for below:

from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes
from Crypto.Util.Padding import pad, unpad


In [17]:
class AESCipher:
    def __init__(self, key):
        self.key = bytes(key)
        self.iv = bytes([47, 28, 226, 107, 181, 219, 38, 58, 118, 181, 139, 61, 227, 13, 96, 202])

    def encrypt(self, data):
        self.cipher = AES.new(self.key, AES.MODE_CBC, self.iv)
        return b64encode(self.iv + self.cipher.encrypt(pad(data.encode('utf-8'), 
            AES.block_size)))

    def decrypt(self, data):
        raw = b64decode(data)
        print(type(raw), raw)
        self.cipher = AES.new(self.key, AES.MODE_CBC, raw[:AES.block_size])
        return unpad(self.cipher.decrypt(raw[AES.block_size:]), AES.block_size)

    def decrypt_raw(self, raw):
        self.cipher = AES.new(self.key, AES.MODE_CBC, self.iv)
        return unpad(self.cipher.decrypt(raw), AES.block_size)


In [18]:
def decrypt(fn):
    # assuming that fn.key (key file) is in the same folder as this script:
    with open(fn + '.key', 'r') as f:
        key = list(map(int, f.readline().split(',')))

    # assuming that fn.locked (locked file) is in the same folder as this script:
    with open(fn + '.locked', 'rb') as f:
        encryptedContent = f.read()
        aes = AESCipher(key)
        orig = aes.decrypt_raw(encryptedContent)
        # and writing to the same folder, too:
        with open(fn, 'wb') as wf:
            wf.write(orig)
    


In [20]:
decrypt('Download.CSV')
decrypt('Download.zip')